In [ ]:
import re
import logging
import numpy as np
import pandas as pd
import multiprocessing
import json
from re import sub
from time import time 
from unidecode import unidecode
from gensim.models import Word2Vec
from collections import defaultdict
from gensim.models import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.models.phrases import Phrases, Phraser

logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [ ]:
df=pd.read_json('yelp_academic_dataset_review.json',lines=True,nrows=300000)

In [ ]:
df2=pd.read_json("yelp_academic_dataset_business (1).json",lines=True)

In [ ]:
file=pd.merge(df, df2, on='business_id', how='inner')
file.head()

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,...,state,postal_code,latitude,longitude,stars_y,review_count,is_open,attributes,categories,hours
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,Turning Point of North Wales,...,PA,19454,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': 'u'average'', 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."
1,VJxlBnJmCDIy8DFG0kjSow,Iaee7y6zdSB3B-kRCo4z1w,XQfwVwDr-v0ZS3_CbbE5Xw,2,0,0,0,This is the second time we tried turning point...,2017-05-13 17:06:55,Turning Point of North Wales,...,PA,19454,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': 'u'average'', 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."
2,S6pQZQocMB1WHMjTRbt77A,ejFxLGqQcWNLdNByJlIhnQ,XQfwVwDr-v0ZS3_CbbE5Xw,4,2,0,1,The place is cute and the staff was very frien...,2017-08-08 00:58:18,Turning Point of North Wales,...,PA,19454,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': 'u'average'', 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."
3,WqgTKVqWVHDHjnjEsBvUgg,f7xa0p_1V9lx53iIGN5Sug,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,We came on a Saturday morning after waiting a ...,2017-11-19 02:20:23,Turning Point of North Wales,...,PA,19454,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': 'u'average'', 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."
4,M0wzFFb7pefOPcxeRVbLag,dCooFVCk8M1nVaQqcfTL3Q,XQfwVwDr-v0ZS3_CbbE5Xw,2,0,0,0,"Mediocre at best. The decor is very nice, and ...",2017-09-09 17:49:47,Turning Point of North Wales,...,PA,19454,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': 'u'average'', 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."


In [ ]:
# file = pd.read_csv("polish_sentiment_dataset.csv")
# file_cleaned = file.dropna().drop_duplicates().reset_index(drop=True)

In [ ]:
file=file[['review_id','business_id','text']]
file_cleaned = file.dropna().drop_duplicates().reset_index(drop=True)

In [ ]:
file_cleaned.head()

,review_id,business_id,text
0,KU_O5udG6zpxOg-VcAEodg,XQfwVwDr-v0ZS3_CbbE5Xw,"If you decide to eat here, just be aware it is..."
1,VJxlBnJmCDIy8DFG0kjSow,XQfwVwDr-v0ZS3_CbbE5Xw,This is the second time we tried turning point...
2,S6pQZQocMB1WHMjTRbt77A,XQfwVwDr-v0ZS3_CbbE5Xw,The place is cute and the staff was very frien...
3,WqgTKVqWVHDHjnjEsBvUgg,XQfwVwDr-v0ZS3_CbbE5Xw,We came on a Saturday morning after waiting a ...
4,M0wzFFb7pefOPcxeRVbLag,XQfwVwDr-v0ZS3_CbbE5Xw,"Mediocre at best. The decor is very nice, and ..."


In [ ]:
# file_cleaned.rate.value_counts()/len(file_cleaned)

In [ ]:
# file_cleaned[file_cleaned.rate==0]

In [ ]:
# file_cleaned = file_cleaned[file_cleaned.rate!=0]

In [ ]:
# file_cleaned.rate.value_counts()/len(file_cleaned)

In [ ]:
def text_to_word_list(text, remove_polish_letters):
    ''' Pre process and convert texts to a list of words 
    method inspired by method from eliorc github repo: https://github.com/eliorc/Medium/blob/master/MaLSTM.ipynb'''
    text = remove_polish_letters(text)
    text = str(text)
    text = text.lower()

    # Clean the text
    text = sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
    text = sub(r"\+", " plus ", text)
    text = sub(r",", " ", text)
    text = sub(r"\.", " ", text)
    text = sub(r"!", " ! ", text)
    text = sub(r"\?", " ? ", text)
    text = sub(r"'", " ", text)
    text = sub(r":", " : ", text)
    text = sub(r"\s{2,}", " ", text)

    text = text.split()

    return text  

In [ ]:
file_cleaned.text = file_cleaned.text.apply(lambda x: text_to_word_list(x, unidecode))

In [ ]:
file_model = file_cleaned.copy()
file_model = file_model[file_model.text.str.len()>1]

In [ ]:
sent = [row for row in file_model.text]
phrases = Phrases(sent, min_count=1, progress_per=50000)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[1]

INFO - 15:05:41: collecting all words and their counts
INFO - 15:05:41: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 15:05:46: PROGRESS: at sentence #50000, processed 5315794 words and 890461 word types
INFO - 15:05:52: PROGRESS: at sentence #100000, processed 10569388 words and 1430856 word types
INFO - 15:05:57: PROGRESS: at sentence #150000, processed 15894822 words and 1896994 word types
INFO - 15:06:03: PROGRESS: at sentence #200000, processed 21125310 words and 2314654 word types
INFO - 15:06:10: PROGRESS: at sentence #250000, processed 26348221 words and 2719098 word types
INFO - 15:06:16: collected 3145940 token types (unigram + bigrams) from a corpus of 31573621 words and 299993 sentences
INFO - 15:06:16: merged Phrases<3145940 vocab, min_count=1, threshold=10.0, max_vocab_size=40000000>
INFO - 15:06:16: Phrases lifecycle event {'msg': 'built Phrases<3145940 vocab, min_count=1, threshold=10.0, max_vocab_size=40000000> in 34.04s', 'datetime': '2022-11-30T

['this',
 'is',
 'the',
 'second',
 'time',
 'we',
 'tried',
 'turning_point',
 'at',
 'this',
 'location',
 'the',
 'first',
 'time',
 'we',
 'had',
 'a',
 'long',
 'wait',
 'for',
 'food',
 'after',
 'ordering',
 'this',
 'time',
 'we',
 'had',
 'an',
 'even',
 'longer',
 'wait',
 'of',
 'over',
 '40_minutes',
 'i',
 'had',
 'the',
 'omelette',
 'skillet',
 'and',
 'there',
 'was',
 'hardly_any',
 'egg',
 'in',
 'it',
 'i',
 'felt_like',
 'i',
 'was',
 'eating',
 'chopped_onions',
 'and',
 'chopped_tomatoes',
 'my',
 'wife',
 'had',
 'a',
 'blt',
 'and',
 'had',
 'a',
 'hard',
 'time',
 'finding',
 'the',
 'tomato',
 'and',
 'the',
 'avocado',
 'that',
 'was',
 'supposed',
 'be',
 'on',
 'it',
 'overall',
 'the',
 'experience',
 'was',
 'stressful',
 'mainly_because',
 'of',
 'the',
 'long',
 'wait']

- min count = 3 - remove most unusual words from training embeddings, like words 'ssssuuuuuuuppppppeeeeeerrrr', which actually stands for 'super', and doesn't need additional training
- window = 4 - Word2Vec model will learn to predict given word from up to 4 words to the left, and up to 4 words to the right
- vector_size = 300 - size of hidden layer used to predict surroundings of embedded word, which also stands for dimensions of trained embeddings
- sample = 1e-5 - probability baseline for subsampling most frequent words from surrounding of embedded word
- negative = 20 - number of negative (ones that shouldn't have been predicted while modeling selected pair of words) words that will have their corresponding weights updated while training on specific training example, along with positive word 

In [ ]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     vector_size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

start = time()

w2v_model.build_vocab(sentences, progress_per=50000)

print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))

INFO - 15:06:22: Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.03>', 'datetime': '2022-11-30T15:06:22.678002', 'gensim': '4.2.0', 'python': '3.9.13 | packaged by conda-forge | (main, May 27 2022, 17:00:33) \n[Clang 13.0.1 ]', 'platform': 'macOS-12.4-arm64-arm-64bit', 'event': 'created'}
INFO - 15:06:22: collecting all words and their counts
INFO - 15:06:22: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 15:06:29: PROGRESS: at sentence #50000, processed 5014215 words, keeping 76960 word types
INFO - 15:06:32: PROGRESS: at sentence #100000, processed 9978328 words, keeping 112729 word types
INFO - 15:06:36: PROGRESS: at sentence #150000, processed 15012098 words, keeping 140468 word types
INFO - 15:06:40: PROGRESS: at sentence #200000, processed 19963118 words, keeping 163685 word types
INFO - 15:06:44: PROGRESS: at sentence #250000, processed 24911338 words, keeping 184473 word types
INFO - 15:06:49: collected 205387 word types

Time to build vocab: 0.53 mins


In [ ]:
start = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=20, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))

w2v_model.init_sims(replace=True)

INFO - 15:07:01: Word2Vec lifecycle event {'msg': 'training model with 7 workers on 92528 vocabulary and 300 features, using sg=0 hs=0 sample=1e-05 negative=20 window=4 shrink_windows=True', 'datetime': '2022-11-30T15:07:01.943258', 'gensim': '4.2.0', 'python': '3.9.13 | packaged by conda-forge | (main, May 27 2022, 17:00:33) \n[Clang 13.0.1 ]', 'platform': 'macOS-12.4-arm64-arm-64bit', 'event': 'train'}
INFO - 15:07:03: EPOCH 0 - PROGRESS: at 0.92% examples, 54820 words/s, in_qsize 5, out_qsize 9
INFO - 15:07:04: EPOCH 0 - PROGRESS: at 3.15% examples, 97997 words/s, in_qsize 10, out_qsize 4
INFO - 15:07:05: EPOCH 0 - PROGRESS: at 5.10% examples, 105833 words/s, in_qsize 13, out_qsize 1
INFO - 15:07:06: EPOCH 0 - PROGRESS: at 7.10% examples, 111997 words/s, in_qsize 9, out_qsize 5
INFO - 15:07:09: EPOCH 0 - PROGRESS: at 9.81% examples, 89325 words/s, in_qsize 14, out_qsize 2
INFO - 15:07:10: EPOCH 0 - PROGRESS: at 13.92% examples, 109135 words/s, in_qsize 14, out_qsize 0
INFO - 15:07:1

INFO - 15:08:29: EPOCH 3 - PROGRESS: at 3.62% examples, 237890 words/s, in_qsize 10, out_qsize 4
INFO - 15:08:30: EPOCH 3 - PROGRESS: at 7.68% examples, 254048 words/s, in_qsize 11, out_qsize 4
INFO - 15:08:31: EPOCH 3 - PROGRESS: at 12.26% examples, 266095 words/s, in_qsize 8, out_qsize 6
INFO - 15:08:32: EPOCH 3 - PROGRESS: at 16.97% examples, 273948 words/s, in_qsize 14, out_qsize 0
INFO - 15:08:33: EPOCH 3 - PROGRESS: at 21.73% examples, 278665 words/s, in_qsize 10, out_qsize 4
INFO - 15:08:34: EPOCH 3 - PROGRESS: at 26.46% examples, 280208 words/s, in_qsize 12, out_qsize 2
INFO - 15:08:35: EPOCH 3 - PROGRESS: at 30.73% examples, 282470 words/s, in_qsize 11, out_qsize 4
INFO - 15:08:36: EPOCH 3 - PROGRESS: at 35.43% examples, 283553 words/s, in_qsize 13, out_qsize 0
INFO - 15:08:37: EPOCH 3 - PROGRESS: at 40.00% examples, 283808 words/s, in_qsize 11, out_qsize 3
INFO - 15:08:38: EPOCH 3 - PROGRESS: at 44.45% examples, 285041 words/s, in_qsize 13, out_qsize 0
INFO - 15:08:39: EPOCH 

INFO - 15:09:53: EPOCH 6 - PROGRESS: at 71.66% examples, 286794 words/s, in_qsize 13, out_qsize 0
INFO - 15:09:54: EPOCH 6 - PROGRESS: at 76.37% examples, 287126 words/s, in_qsize 9, out_qsize 5
INFO - 15:09:55: EPOCH 6 - PROGRESS: at 80.93% examples, 287817 words/s, in_qsize 11, out_qsize 1
INFO - 15:09:56: EPOCH 6 - PROGRESS: at 85.35% examples, 287768 words/s, in_qsize 13, out_qsize 3
INFO - 15:09:57: EPOCH 6 - PROGRESS: at 89.96% examples, 288403 words/s, in_qsize 13, out_qsize 3
INFO - 15:09:58: EPOCH 6 - PROGRESS: at 94.51% examples, 289258 words/s, in_qsize 13, out_qsize 0
INFO - 15:09:59: EPOCH 6 - PROGRESS: at 98.88% examples, 290035 words/s, in_qsize 14, out_qsize 2
INFO - 15:10:00: EPOCH 6: training on 29868966 raw words (6608201 effective words) took 22.7s, 291517 effective words/s
INFO - 15:10:01: EPOCH 7 - PROGRESS: at 3.62% examples, 238995 words/s, in_qsize 11, out_qsize 5
INFO - 15:10:02: EPOCH 7 - PROGRESS: at 7.83% examples, 256010 words/s, in_qsize 11, out_qsize 5
I

INFO - 15:11:19: EPOCH 10 - PROGRESS: at 27.98% examples, 168766 words/s, in_qsize 10, out_qsize 7
INFO - 15:11:20: EPOCH 10 - PROGRESS: at 32.77% examples, 180928 words/s, in_qsize 13, out_qsize 0
INFO - 15:11:21: EPOCH 10 - PROGRESS: at 37.02% examples, 188294 words/s, in_qsize 9, out_qsize 5
INFO - 15:11:22: EPOCH 10 - PROGRESS: at 41.49% examples, 196100 words/s, in_qsize 13, out_qsize 0
INFO - 15:11:23: EPOCH 10 - PROGRESS: at 45.87% examples, 202285 words/s, in_qsize 10, out_qsize 2
INFO - 15:11:24: EPOCH 10 - PROGRESS: at 50.22% examples, 207414 words/s, in_qsize 14, out_qsize 0
INFO - 15:11:25: EPOCH 10 - PROGRESS: at 54.56% examples, 211867 words/s, in_qsize 13, out_qsize 0
INFO - 15:11:26: EPOCH 10 - PROGRESS: at 59.11% examples, 215984 words/s, in_qsize 12, out_qsize 6
INFO - 15:11:27: EPOCH 10 - PROGRESS: at 63.56% examples, 219920 words/s, in_qsize 10, out_qsize 6
INFO - 15:11:28: EPOCH 10 - PROGRESS: at 68.01% examples, 223305 words/s, in_qsize 11, out_qsize 3
INFO - 15:1

INFO - 15:14:41: EPOCH 13 - PROGRESS: at 38.43% examples, 246844 words/s, in_qsize 11, out_qsize 0
INFO - 15:14:42: EPOCH 13 - PROGRESS: at 41.78% examples, 244935 words/s, in_qsize 13, out_qsize 0
INFO - 15:14:43: EPOCH 13 - PROGRESS: at 45.18% examples, 243041 words/s, in_qsize 13, out_qsize 0
INFO - 15:14:44: EPOCH 13 - PROGRESS: at 49.23% examples, 244410 words/s, in_qsize 11, out_qsize 3
INFO - 15:14:45: EPOCH 13 - PROGRESS: at 53.49% examples, 246170 words/s, in_qsize 10, out_qsize 4
INFO - 15:14:46: EPOCH 13 - PROGRESS: at 57.77% examples, 247373 words/s, in_qsize 12, out_qsize 2
INFO - 15:14:48: EPOCH 13 - PROGRESS: at 61.84% examples, 248335 words/s, in_qsize 13, out_qsize 1
INFO - 15:14:49: EPOCH 13 - PROGRESS: at 65.89% examples, 249105 words/s, in_qsize 13, out_qsize 0
INFO - 15:14:50: EPOCH 13 - PROGRESS: at 69.92% examples, 249511 words/s, in_qsize 12, out_qsize 2
INFO - 15:14:51: EPOCH 13 - PROGRESS: at 74.05% examples, 250453 words/s, in_qsize 14, out_qsize 0
INFO - 15:

INFO - 15:16:04: EPOCH 16 - PROGRESS: at 78.83% examples, 282455 words/s, in_qsize 11, out_qsize 0
INFO - 15:16:05: EPOCH 16 - PROGRESS: at 83.16% examples, 282094 words/s, in_qsize 13, out_qsize 1
INFO - 15:16:06: EPOCH 16 - PROGRESS: at 87.50% examples, 282776 words/s, in_qsize 13, out_qsize 1
INFO - 15:16:07: EPOCH 16 - PROGRESS: at 92.16% examples, 284232 words/s, in_qsize 11, out_qsize 1
INFO - 15:16:08: EPOCH 16 - PROGRESS: at 96.60% examples, 285271 words/s, in_qsize 11, out_qsize 0
INFO - 15:16:09: EPOCH 16: training on 29868966 raw words (6613715 effective words) took 23.1s, 286728 effective words/s
INFO - 15:16:10: EPOCH 17 - PROGRESS: at 3.45% examples, 223501 words/s, in_qsize 13, out_qsize 4
INFO - 15:16:11: EPOCH 17 - PROGRESS: at 7.43% examples, 245759 words/s, in_qsize 12, out_qsize 3
INFO - 15:16:12: EPOCH 17 - PROGRESS: at 12.01% examples, 259173 words/s, in_qsize 8, out_qsize 8
INFO - 15:16:13: EPOCH 17 - PROGRESS: at 16.77% examples, 267848 words/s, in_qsize 10, out

/var/folders/1j/vhrxwjz94tn0tsz00qlnqw8m0000gn/T/ipykernel_10709/258766345.py:7: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)
WARNING - 15:21:00: destructive init_sims(replace=True) deprecated & no longer required for space-efficiency


Time to train the model: 13.98 mins


In [ ]:
w2v_model.save("word2vec.model")

INFO - 15:21:00: Word2Vec lifecycle event {'fname_or_handle': 'word2vec.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-11-30T15:21:00.957790', 'gensim': '4.2.0', 'python': '3.9.13 | packaged by conda-forge | (main, May 27 2022, 17:00:33) \n[Clang 13.0.1 ]', 'platform': 'macOS-12.4-arm64-arm-64bit', 'event': 'saving'}
INFO - 15:21:00: storing np array 'vectors' to word2vec.model.wv.vectors.npy
INFO - 15:21:01: storing np array 'syn1neg' to word2vec.model.syn1neg.npy
INFO - 15:21:01: not storing attribute cum_table
INFO - 15:21:01: saved word2vec.model


Exporting preprocessed dataset for further steps (with replaced bigrams)

In [ ]:
file_export = file_model.copy()
file_export['old_text'] = file_export.text
file_export.old_text = file_export.old_text.str.join(' ')
file_export.text = file_export.text.apply(lambda x: ' '.join(bigram[x]))
# file_export.rate = file_export.rate.astype('int8')

In [ ]:
file_export[['text', 'business_id','review_id']].to_csv('cleaned_dataset.csv', index=False)